In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.conv1_1 = nn.Conv2d(1, 8, 3, padding=1, bias=False) #input: 28 Output: 28 RF_out: 3
        self.relu1_1 = nn.ReLU()
        self.bn1_1 = nn.BatchNorm2d(8)
        self.conv1_2 = nn.Conv2d(8, 8, 3, padding=1, bias=False) #input: 28 Output: 28 RF_out: 5
        self.relu1_2 = nn.ReLU()
        self.bn1_2 = nn.BatchNorm2d(8)
        self.conv1_3 = nn.Conv2d(8, 8, 3, padding=1, bias=False) #input: 28 Output: 28 RF_out: 7
        self.relu1_3 = nn.ReLU()
        self.bn1_3 = nn.BatchNorm2d(8)
        self.maxpool1 = nn.MaxPool2d(2, 2) #input: 28 Output: 14 RF_out: 9
        self.dropout1 = nn.Dropout(0.25)

        self.conv2_1 = nn.Conv2d(8, 16, 3, padding=1, bias=False) #input: 14 Output: 14 RF_out: 13
        self.relu2_1 = nn.ReLU()
        self.bn2_1 = nn.BatchNorm2d(16)
        self.conv2_2 = nn.Conv2d(16, 16, 3, padding=1, bias=False) #input: 14 Output: 14 RF_out: 17
        self.relu2_2 = nn.ReLU()
        self.bn2_2 = nn.BatchNorm2d(16)
        self.conv2_3 = nn.Conv2d(16, 16, 3, padding=1, bias=False) #input: 14 Output: 14 RF_out: 21
        self.relu2_3 = nn.ReLU()
        self.bn2_3 = nn.BatchNorm2d(16)
        self.maxpool2 = nn.MaxPool2d(2, 2) #input: 14 Output: 7 RF_out: 23
        self.dropout2 = nn.Dropout(0.25)

        self.conv3_1 = nn.Conv2d(16, 16, 3, padding=1, bias=False) #input: 7 Output: 7 RF_out: 31
        self.relu3_1 = nn.ReLU()
        self.bn3_1 = nn.BatchNorm2d(16)
        self.conv3_2 = nn.Conv2d(16, 16, 3, bias=False) #input: 7 Output: 5 RF_out: 35
        self.relu3_2 = nn.ReLU()
        self.conv3_3 = nn.Conv2d(16, 16, 3, bias=False) #input: 5 Output: 3 RF_out: 39
        self.relu3_3 = nn.ReLU()
        self.conv3_4 = nn.Conv2d(16, 32, 3, bias=False) #input: 3 Output: 1 RF_out: 43

        self.fc = nn.Conv2d(32, 10, 1)

    def forward(self, x):
        x = self.dropout1(self.maxpool1(self.bn1_3(self.relu1_3(self.conv1_3(self.bn1_2(self.relu1_2(self.conv1_2(self.bn1_1(self.relu1_1(self.conv1_1(x)))))))))))
        x = self.dropout2(self.maxpool2(self.bn2_3(self.relu2_3(self.conv2_3(self.bn2_2(self.relu2_2(self.conv2_2(self.bn2_1(self.relu2_1(self.conv2_1(x)))))))))))
        x = self.conv3_4(self.relu3_3(self.conv3_3(self.relu3_2(self.conv3_2(self.bn3_1(self.relu3_1(self.conv3_1(x))))))))
        x = F.avg_pool2d(x, x.shape[2:])
        x = self.fc(x)
        x = x.view(-1, 10)
        x = F.log_softmax(x, dim=1)
        return x

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
              ReLU-2            [-1, 8, 28, 28]               0
       BatchNorm2d-3            [-1, 8, 28, 28]              16
            Conv2d-4            [-1, 8, 28, 28]             576
              ReLU-5            [-1, 8, 28, 28]               0
       BatchNorm2d-6            [-1, 8, 28, 28]              16
            Conv2d-7            [-1, 8, 28, 28]             576
              ReLU-8            [-1, 8, 28, 28]               0
       BatchNorm2d-9            [-1, 8, 28, 28]              16
        MaxPool2d-10            [-1, 8, 14, 14]               0
          Dropout-11            [-1, 8, 14, 14]               0
           Conv2d-12           [-1, 16, 14, 14]           1,152
             ReLU-13           [-1, 16, 14, 14]               0
      BatchNorm2d-14           [-1, 16,

In [ ]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


100%|██████████| 9912422/9912422 [00:00<00:00, 90637144.23it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 68283931.13it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 25136082.24it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 14454118.94it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.05167656019330025 batch_id=468: 100%|██████████| 469/469 [01:24<00:00,  5.57it/s]



Test set: Average loss: 0.0575, Accuracy: 9826/10000 (98%)



loss=0.02505149506032467 batch_id=468: 100%|██████████| 469/469 [01:18<00:00,  6.00it/s]



Test set: Average loss: 0.0436, Accuracy: 9861/10000 (99%)



loss=0.07747160643339157 batch_id=468: 100%|██████████| 469/469 [01:22<00:00,  5.72it/s]


...


Test set: Average loss: 0.0192, Accuracy: 9938/10000 (99%)



loss=0.0519535057246685 batch_id=468: 100%|██████████| 469/469 [01:17<00:00,  6.02it/s]



Test set: Average loss: 0.0178, Accuracy: 9946/10000 (99%)



loss=0.0007642873097211123 batch_id=468: 100%|██████████| 469/469 [01:17<00:00,  6.03it/s]



Test set: Average loss: 0.0202, Accuracy: 9934/10000 (99%)

